## Librerías



In [11]:
from bs4 import BeautifulSoup
import requests
import matplotlib
import re
from tqdm.notebook import tnrange #vamos a trackear el progreso
import pandas as pd

## Recopilando información de la web


In [20]:
def get_all_links(url):
    '''Obtención de todos los Links'''

    headers = {'User-Agent': 'Mozilla/5.0'}
    r = requests.get(url, headers=headers)
    print(r)
    soup = BeautifulSoup(r.text, 'html.parser')
    search = soup.find_all('div', class_='timebucket u-inlineBlock u-width50')
    years = []
    for h in search:
        print(f"esto es la h del primer for {h}")
        years.append(h.a.get('href'))
    print("Total de años en:  "  + str(len(years)))

    # Ahora los meses
    years_months = []
    years_no_months = [] #para los años que no tienen meses
    for year in years:
        y_soup = BeautifulSoup(requests.get(year, headers=headers).text, 'html.parser')
        search_month = y_soup.find_all('div', class_='timebucket u-inlineBlock u-width80')
        months = []
        if search_month:
            for month in search_month:
                try:
                    months.append(month.a.get('href'))
                except:
                    pass
            years_months.append(months)
        else:
            years_no_months.append(year)
    years_months = [item for sublist in years_months for item in sublist]
    print("Total de años con meses "  + str(len(years_months)))
    print("Total de años sin meses "  + str(len(years_no_months)))
    # a por los días
    all_links = []
    all_links_no_days = [] #Para los meses que no tengan días
    for month_url in years_months:
        print(month_url)
        m_soup = BeautifulSoup(requests.get(month_url).text, 'html.parser')
        all_days = m_soup.find_all('div', class_='timebucket u-inlineBlock u-width35')
        days = []
        if all_days:
            for day in all_days:
                try:
                    days.append(day.a.get('href'))
                except:
                    pass
            all_links.append(days)
        else:
            all_links_no_days.append(month_url)
    all_links = [item for sublist in all_links for item in sublist]
    final_links = years_no_months+all_links_no_days+all_links
    return final_links
towards_ai_links = get_all_links('https://towardsdatascience.com/archive')


<Response [200]>
esto es la h del primer for <div class="timebucket u-inlineBlock u-width50"><a href="https://towardsdatascience.com/archive/2010">2010</a></div>
esto es la h del primer for <div class="timebucket u-inlineBlock u-width50"><a href="https://towardsdatascience.com/archive/2011">2011</a></div>
esto es la h del primer for <div class="timebucket u-inlineBlock u-width50"><a href="https://towardsdatascience.com/archive/2013">2013</a></div>
esto es la h del primer for <div class="timebucket u-inlineBlock u-width50"><a href="https://towardsdatascience.com/archive/2014">2014</a></div>
esto es la h del primer for <div class="timebucket u-inlineBlock u-width50"><a href="https://towardsdatascience.com/archive/2015">2015</a></div>
esto es la h del primer for <div class="timebucket u-inlineBlock u-width50"><a href="https://towardsdatascience.com/archive/2016">2016</a></div>
esto es la h del primer for <div class="timebucket u-inlineBlock u-width50"><a href="https://towardsdatascience.c

## Prueba pequeña de una URL

In [23]:
headers = {'User-Agent': 'Mozilla/5.0'}
link = 'https://towardsdatascience.com/archive/2023/02'
soup_ = BeautifulSoup(requests.get(link, headers=headers).text, 'html.parser')
articles = soup_.find_all('div', class_= 'streamItem streamItem--postPreview js-streamItem')

article = articles[0] #Coge un solo artículo
if article.h3:
    title = article.h3.getText()
elif article.h2:
    title = article.h2.getText()
else:
    title = ''
if article.h4:
    subtitle = article.h4.getText()
else:
    subtitle = ''


## Estracción de comentarios y Likes

In [24]:
# Si el artículo tiene respuestas de los lectores,
# viene en un listado en este caso, así que lo sacaremos con el número que tiene la respuesta
# 1, respuesta 2, respuesta...

s_clap = article.find('button', class_='button button--chromeless u-baseColor--buttonNormal js-multirecommendCountButton u-disablePointerEvents')

if s_clap:
    s_clap = s_clap.getText()
    print(s_clap)
    if 'K' in s_clap:
        clap = int(float(s_clap.replace('K', ''))*1000)
        print(f"esto es el s_clap {clap}")
    else:
        clap = int(s_clap)
else:
    clap = 0

s_response = article.find('a', class_='button button--chromeless u-baseColor--buttonNormal')
if s_response:
    s_response = s_response.getText()
    response = int(re.search(r'\d+', s_response).group())
else:
    response = 0

1.4K
esto es el s_clap 1400


## Extracción en bulk de los contenidos

In [28]:
def get_data_all_articles(final_links):
    titles, sub_titles, claps, responses = [], [], [], []

    for link, z in zip(final_links, tnrange(len(final_links))):
        headers = {'User-Agent': 'Mozilla/5.0'}
        soup_ = BeautifulSoup(requests.get(link, headers=headers).text, 'html.parser')
        articles = soup_.find_all('div', class_='streamItem streamItem--postPreview js-streamItem')
        title, subtitle, clap, response = [], [], [], []
        for article in articles:
            if article.h3:
                title.append(article.h3.getText())
            elif article.h2:
                title.append(article.h2.getText())
            else:
                title.append('')
            if article.h4:
                subtitle.append(article.h4.getText())
            else:
                subtitle.append('')
            s_clap = article.find('button', class_= 'button button--chromeless u-baseColor--buttonNormal js-multirecommendCountButton u-disablePointerEvents')
            if s_clap:
                s_clap = s_clap.getText()
                if 'K' in s_clap:
                    clap.append(int(float(s_clap.replace('K', '')))*1000)
                else:
                    clap.append(int(s_clap))
            else:
                clap.append(0)
            s_response =article.find('a', class_='button button--chromeless u-baseColor--buttonNormal')
            if s_response:
                s_response = s_response.getText()
                response.append(int(re.search(r'\d+', s_response).group()))
            else:
                response.append(0)
        titles.append(title)
        sub_titles.append(subtitle)
        claps.append(clap)
        responses.append(response)
    titles = [item for sublist in titles for item in sublist]
    sub_titles = [item for sublist in sub_titles for item in sublist]
    claps = [item for sublist in claps for item in sublist]
    responses = [item for sublist in responses for item in sublist]
    frame = pd.DataFrame([titles, sub_titles, claps, responses]).transpose()
    frame.columns = ['Title','Subtitle','claps','Responses']
    return frame


In [29]:
data_set = get_data_all_articles(towards_ai_links)
print(data_set)

  0%|          | 0/114 [00:00<?, ?it/s]

                                                 Title  \
0             Avoiding an Epic Data Visualization Fail   
1           What makes a good data scientist-engineer?   
2    Lessons Learned From The Higgs Boson Kaggle Ch...   
3                           How Do You Flowchart Code?   
4    Data Engineer vs Data Scientist vs Business An...   
..                                                 ...   
990                               The Math Behind LSTM   
991               My First Billion (of Rows) in DuckDB   
992  Four Simple Steps to Build a Custom Self-Hoste...   
993  Breaking down State-of-the-Art PPO Implementat...   
994            Tool Use, Agents, and the Voyager Paper   

                                              Subtitle claps Responses  
0                    It’s simple — answer the question   181         1  
1    The qualities you need to develop to make an i...   165         0  
2                                                          4         0  
3       I’m

## Esportador de contenido

In [30]:
data_set.to_csv('tds.csv')